In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor

In [3]:
from sklearn.metrics import r2_score as r2

In [4]:
test = pd.read_csv("C:/Users/Acer/Desktop/Kaggle/Big Mart sales data/Test.csv")

In [5]:
train = pd.read_csv("C:/Users/Acer/Desktop/Kaggle/Big Mart sales data/Train.csv")

In [6]:
train['Item_Outlet_Sales'].describe()

count     8523.000000
mean      2181.288914
std       1706.499616
min         33.290000
25%        834.247400
50%       1794.331000
75%       3101.296400
max      13086.964800
Name: Item_Outlet_Sales, dtype: float64

In [7]:
y = train['Item_Outlet_Sales']

In [8]:
train = train.drop('Item_Outlet_Sales', axis =1)

In [9]:
len(train)

8523

In [10]:
ds = pd.concat([train, test], axis = 0)

In [11]:
ds.shape

(14204, 11)

In [12]:
ds.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
count,11765.000000,14204.000000,14204.000000,14204.000000
mean,12.792854,0.065953,141.004977,1997.830681
std,4.652502,0.051459,62.086938,8.371664
min,4.555000,0.000000,31.290000,1985.000000
25%,8.710000,0.027036,94.012000,1987.000000
50%,12.600000,0.054021,142.247000,1999.000000
75%,16.750000,0.094037,185.855600,2004.000000
max,21.350000,0.328391,266.888400,2009.000000


In [13]:
ds = ds.drop('Item_Outlet_Sales', axis =  1)  ## this is the y variable

KeyError: "['Item_Outlet_Sales'] not found in axis"

In [ ]:
ds['Item_Weight'] = ds['Item_Weight'].fillna(ds['Item_Weight'].mean())

In [ ]:
ds['Item_Weight'].describe()

In [ ]:
ds.isna().sum()

In [ ]:
le = LabelEncoder()

In [ ]:
ds.dtypes

In [14]:
cat_cols = ds.select_dtypes(include='object').columns

print(cat_cols)

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')


In [15]:
for col in cat_cols:
    le = LabelEncoder()
    ds[col] = le.fit_transform(ds[col])

In [16]:
ds.tail()        ### great

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
5676,233,10.5,2,0.013496,13,141.3154,8,1997,2,0,1
5677,308,7.6,2,0.142991,15,169.1448,3,2009,1,2,2
5678,1426,10.0,1,0.073529,8,118.7440,7,2002,3,1,1
5679,521,15.3,2,0.000000,3,214.6218,2,2007,3,1,1
5680,997,9.5,2,0.104720,3,79.7960,7,2002,3,1,1


In [17]:
train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
count,7060.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867
std,4.643456,0.051598,62.275067,8.371760
min,4.555000,0.000000,31.290000,1985.000000
25%,8.773750,0.026989,93.826500,1987.000000
50%,12.600000,0.053931,143.012800,1999.000000
75%,16.850000,0.094585,185.643700,2004.000000
max,21.350000,0.328391,266.888400,2009.000000


In [18]:
test.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
count,4705.000000,5681.000000,5681.000000,5681.000000
mean,12.695633,0.065684,141.023273,1997.828903
std,4.664849,0.051252,61.809091,8.372256
min,4.555000,0.000000,31.990000,1985.000000
25%,8.645000,0.027047,94.412000,1987.000000
50%,12.500000,0.054154,141.415400,1999.000000
75%,16.700000,0.093463,186.026600,2004.000000
max,21.350000,0.323637,266.588400,2009.000000


### Now I will break ds into train and test

In [19]:
train1 = ds.iloc[:len(train)]
test1 = ds.iloc[len(train1):]

In [20]:
train1.describe()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,8523.000000,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000,8523.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,779.714889,12.857645,1.369354,0.066132,7.226681,140.992782,4.722281,1997.831867,1.736360,1.112871,1.201220
std,449.222377,4.643456,0.644810,0.051598,4.209990,62.275067,2.837201,8.371760,0.989181,0.812757,0.796459
min,0.000000,4.555000,0.000000,0.000000,0.000000,31.290000,0.000000,1985.000000,0.000000,0.000000,0.000000
25%,395.500000,8.773750,1.000000,0.026989,4.000000,93.826500,2.000000,1987.000000,1.000000,0.000000,1.000000
50%,783.000000,12.600000,1.000000,0.053931,6.000000,143.012800,5.000000,1999.000000,2.000000,1.000000,1.000000
75%,1167.000000,16.850000,2.000000,0.094585,10.000000,185.643700,7.000000,2004.000000,3.000000,2.000000,1.000000
max,1558.000000,21.350000,4.000000,0.328391,15.000000,266.888400,9.000000,2009.000000,3.000000,2.000000,3.000000


In [21]:
test1.describe()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,5681.000000,4705.000000,5681.000000,5681.000000,5681.000000,5681.000000,5681.000000,5681.000000,5681.000000,5681.000000,5681.000000
mean,780.912515,12.695633,1.368773,0.065684,7.233410,141.023273,4.722760,1997.828903,1.736314,1.112832,1.201373
std,450.253632,4.664849,0.641085,0.051252,4.219057,61.809091,2.837201,8.372256,0.989113,0.812824,0.796740
min,0.000000,4.555000,0.000000,0.000000,0.000000,31.990000,0.000000,1985.000000,0.000000,0.000000,0.000000
25%,384.000000,8.645000,1.000000,0.027047,4.000000,94.412000,2.000000,1987.000000,1.000000,0.000000,1.000000
50%,779.000000,12.500000,1.000000,0.054154,6.000000,141.415400,5.000000,1999.000000,2.000000,1.000000,1.000000
75%,1173.000000,16.700000,2.000000,0.093463,10.000000,186.026600,7.000000,2004.000000,3.000000,2.000000,1.000000
max,1558.000000,21.350000,4.000000,0.323637,15.000000,266.588400,9.000000,2009.000000,3.000000,2.000000,3.000000


### Time to create the models

In [22]:
xgb = XGBRegressor()

In [25]:
X = train1

In [26]:
xgb.fit(X,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [27]:
y_xgb = xgb.predict(X)

In [29]:
r2(y_xgb, y)            ### 78.1 % accuracy on train dataset

0.7817751140114899

In [30]:
test1.describe()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,5681.000000,4705.000000,5681.000000,5681.000000,5681.000000,5681.000000,5681.000000,5681.000000,5681.000000,5681.000000,5681.000000
mean,780.912515,12.695633,1.368773,0.065684,7.233410,141.023273,4.722760,1997.828903,1.736314,1.112832,1.201373
std,450.253632,4.664849,0.641085,0.051252,4.219057,61.809091,2.837201,8.372256,0.989113,0.812824,0.796740
min,0.000000,4.555000,0.000000,0.000000,0.000000,31.990000,0.000000,1985.000000,0.000000,0.000000,0.000000
25%,384.000000,8.645000,1.000000,0.027047,4.000000,94.412000,2.000000,1987.000000,1.000000,0.000000,1.000000
50%,779.000000,12.500000,1.000000,0.054154,6.000000,141.415400,5.000000,1999.000000,2.000000,1.000000,1.000000
75%,1173.000000,16.700000,2.000000,0.093463,10.000000,186.026600,7.000000,2004.000000,3.000000,2.000000,1.000000
max,1558.000000,21.350000,4.000000,0.323637,15.000000,266.588400,9.000000,2009.000000,3.000000,2.000000,3.000000


In [32]:
y_test = xgb.predict(test1)        #### first prediction for test set

In [33]:
ls = Lasso()

In [ ]:
lr.foit